In [19]:
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import text
dataset_file = '/home/ubuntu/workspace/XMODE/dataset/mimic_iv_cxr/sampled_test_with_scope_preprocessed_balenced_answer_100.json'

def get_image_id_by_study_id(study_id, session):
    query = f"SELECT image_id FROM tb_cxr WHERE study_id = {study_id}"
    result = session.execute(text(query))
    image_id = [row[0] for row in result]
    return image_id

def get_subject_id_by_study_id(study_id, session):
    query = f"SELECT subject_id FROM tb_cxr WHERE study_id = {study_id}"
    result = session.execute(text(query))
    subject_id = [row[0] for row in result]
    return subject_id

def get_image_ids_by_patient_ids(patient_id, session):
    query = f"SELECT image_id FROM tb_cxr WHERE subject_id = {patient_id}"
    result = session.execute(text(query))
    image_ids = []
    for row in result:
        image_ids.append(row[0])
    return image_ids

with open(dataset_file, 'r') as f:
    dataset = json.load(f)

In [20]:

Base = declarative_base()
engine = create_engine('sqlite:////home/ubuntu/workspace/XMODE/mimic_iv_cxr.db')
Session = sessionmaker(bind=engine)
session = Session()

all_images = []
all_patients = []
for d in dataset:
    for k, v in d["value"].items():
        image_ids = []
        if k == "study_id":
            image_ids = get_image_id_by_study_id(v, session)
            patient_id = get_subject_id_by_study_id(v, session)
            all_patients.extend(patient_id)
        elif k == "patient_id":
            image_ids = get_image_ids_by_patient_ids(v, session)
            all_patients.extend([v])
        all_images.extend(image_ids)

all_images = list(set(all_images))
print("all images", len(all_images))
all_patients = list(set(all_patients))
print("all patients", len(all_patients))

all images 432
all patients 78


/tmp/ipykernel_3861/3954809438.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
